# Assignment #3: A simple language classifier with scikit-learn and Keras

Author: Pierre Nugues

## Objectives

In this assignment, you will implement a language detector inspired from Google's _Compact language detector_, version 3 (CLD3): https://github.com/google/cld3. CLD3 is written in C++ and its code is available from GitHub. The objectives of the assignment are to:
* Write a program to classify languages
* Use neural networks with sklearn and either Keras or PyTorch
* Know what a classifier is
* Write a short report of 1 to 2 pages to describe your program. You will notably comment the performance you obtained and how you could improve it.

## Description

### System Overview

Read the GitHub description of CLD3, https://github.com/google/cld3, (_Model_ section). In your individual report you will:
1. Summarize the system in two or three sentences;
2. Outline the CLD3 overall architecture in a figure. Use building blocks only and do not specify the parameters.

## Imports

In [1]:
import bz2
import json
import os
import numpy as np
import requests
import sys
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import confusion_matrix

### Dataset

As dataset, we will use Tatoeba, https://tatoeba.org/eng/downloads. It consists of more than 8 million short texts in 347 languages and it is available in one file called `sentences.csv`.

The dataset is structured this way: There is one text per line, where each line consists of the three following fields separated by tabulations and ended by a carriage return:
```
sentence id [tab] language code [tab] text [cr]
```
Each text (sentence) has a unique id and has a language code that follows the ISO 639-3 standard (see below). 

### Scope of the lab

In this lab, you will consider three languages only: French (fra), English (eng), and Swedish (swe). Below is an excerpt of the Tatoeba dataset limited to these three languages: 

```
1276    eng     Let's try something.
1277    eng     I have to go to sleep.
1280    eng     Today is June 18th and it is Muiriel's birthday!
...
1115    fra     Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.
1279    fra     Je ne supporte pas ce type.
1441    fra     Pour une fois dans ma vie je fais un bon geste... Et ça ne sert à rien.
...
337413  swe     Vi trodde att det var ett flygande tefat.
341910  swe     Detta är huset jag bodde i när jag var barn.
341938  swe     Vi hade roligt på stranden igår.
...
```
Tatoeba is updated continuously. The examples from this dataset come from a corpus your instructor downloaded on September 23, 2021.

### Understanding the $\mathbf{X}$ matrix (feature matrix)

You will now investigate the CLD3 features:
 *  What are the features CLD3 extracts from each text?
 * Create manually a simplified $\mathbf{X}$ matrix where you will represent the 9 texts with CLD3 features. You will use a restricted set of features: You will only consider the letters _a_, _b_, and _n_ and the bigrams _an_, _ba_, and _na_. You will ignore the the rest of letters and bigrams as well as the trigrams. Your matrix will have 9 rows and 6 columns, each column will contain these counts: `[#a, #b, #n, #an, #ba, #na]`.

The CLD3's original description uses relative frequencies (counts of a letter divided by the total counts of letters in the text). Here, you will use the raw counts. To help you, your instructor filled the fourth row of the matrix corresponding to the first text in French. Fill in the rest. You will include this matrix in your report. 

$\mathbf{X} =
\begin{bmatrix}
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
8& 0& 8& 1&0&0\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
     \text{eng} \\
     \text{eng}\\
     \text{eng}\\
    \text{fra}\\
   \text{fra}  \\
     \text{fra}\\
    \text{swe}\\
 \text{swe}   \\
 \text{swe}   
\end{bmatrix}$

## Programming: Extracting the features

Before you start programming, download the Tatoeba dataset. You can use the intructions:

In [2]:
!wget https://downloads.tatoeba.org/exports/sentences.tar.bz2

--2022-09-16 10:04:24--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167951047 (160M) [application/octet-stream]
Saving to: ‘sentences.tar.bz2.1’

sentences.tar.bz2.1 100%[===================>] 160,17M  32,4MB/s    in 5,7s    

2022-09-16 10:04:31 (28,2 MB/s) - ‘sentences.tar.bz2.1’ saved [167951047/167951047]



In [3]:
!tar -xvjf sentences.tar.bz2

x sentences.csv


### Loading and filtering the dataset

Run the code to read the dataset and split it into lines. You may have to change the path

In [4]:
dataset = open('sentences.csv', encoding='utf8').read().strip()
dataset = dataset.split('\n')
dataset[:10]

['1\tcmn\t我們試試看！',
 '2\tcmn\t我该去睡觉了。',
 '3\tcmn\t你在干什麼啊？',
 '4\tcmn\t這是什麼啊？',
 '5\tcmn\t今天是６月１８号，也是Muiriel的生日！',
 '6\tcmn\t生日快乐，Muiriel！',
 '7\tcmn\tMuiriel现在20岁了。',
 '8\tcmn\t密码是"Muiriel"。',
 '9\tcmn\t我很快就會回來。',
 '10\tcmn\t我不知道。']

Run the code to split the fields and remove possible whitespaces

In [5]:
dataset = list(map(lambda x: tuple(x.split('\t')), dataset))
dataset = list(map(lambda x: tuple(map(str.strip, x)), dataset))
dataset[:3]

[('1', 'cmn', '我們試試看！'), ('2', 'cmn', '我该去睡觉了。'), ('3', 'cmn', '你在干什麼啊？')]

Write the code to extract the French, English, and Swedish texts. You will call the resulting dataset: `dataset_small`

In [6]:
# Write your code here
dataset_small = [x for x in dataset if x[1] in {'eng', 'fra', 'swe'}]

In [7]:
dataset_small[:5]

[('1115',
  'fra',
  "Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent."),
 ('1276', 'eng', "Let's try something."),
 ('1277', 'eng', 'I have to go to sleep.'),
 ('1279', 'fra', 'Je ne supporte pas ce type.'),
 ('1280', 'eng', "Today is June 18th and it is Muiriel's birthday!")]

### Functions to Count Characters Ngrams

Write a function `count_chars(string, lc=True)` to count characters (unigrams) of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the unigrams.

In [24]:
# Write your code here
from collections import Counter
def count_chars(string, lc=True):
    if lc:
        string = string.lower()
    counts = Counter(string)
    sum_counts = sum(Counter(string).values())
    print(counts)
    return [c for c in counts]

In [25]:
count_chars("Let's try something.")

Counter({'t': 3, 'e': 2, 's': 2, ' ': 2, 'l': 1, "'": 1, 'r': 1, 'y': 1, 'o': 1, 'm': 1, 'h': 1, 'i': 1, 'n': 1, 'g': 1, '.': 1})


['l', 'e', 't', "'", 's', ' ', 'r', 'y', 'o', 'm', 'h', 'i', 'n', 'g', '.']

Write a function `count_bigrams(string, lc=True)` to count the characters bigrams of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the bigrams.

In [10]:
# Write your code here


Write a function `count_trigrams(string, lc=True)` to count the characters trigrams of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the trigrams.

In [11]:
# Write your code here


In [12]:
count_chars("Let's try something.")

Counter({'l': 1,
         'e': 2,
         't': 3,
         "'": 1,
         's': 2,
         ' ': 2,
         'r': 1,
         'y': 1,
         'o': 1,
         'm': 1,
         'h': 1,
         'i': 1,
         'n': 1,
         'g': 1,
         '.': 1})

In [13]:
count_bigrams("Let's try something.")

NameError: name 'count_bigrams' is not defined

In [ ]:
count_trigrams("Let's try something.")

### Counting the ngrams in the dataset

You will now extract the features from each text. For this, add the character, bigram, and trigram relative frequencies to the texts using this format:
`(text_id, language_id, text, char_cnt, bigram_cnt, trigram_cnt)`.

From the datapoint:
`('1276', 'eng', "Let's try something.")`,
you must return:

`('1276', 'eng', "Let's try something.", 
  {'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 'n': 0.05, 'g': 0.05, '.': 0.05},
  {'le': 0.05263157894736842, 'et': 0.10526315789473684, "t'": 0.05263157894736842, "'s": 0.05263157894736842, 's ': 0.05263157894736842, ' t': 0.05263157894736842, 'tr': 0.05263157894736842, 'ry': 0.05263157894736842, 'y ': 0.05263157894736842, ' s': 0.05263157894736842, 'so': 0.05263157894736842, 'om': 0.05263157894736842, 'me': 0.05263157894736842, 'th': 0.05263157894736842, 'hi': 0.05263157894736842, 'in': 0.05263157894736842, 'ng': 0.05263157894736842, 'g.': 0.05263157894736842},
  {'let': 0.05555555555555555, "et'": 0.05555555555555555, "t's": 0.05555555555555555, "'s ": 0.05555555555555555, 's t': 0.05555555555555555, ' tr': 0.05555555555555555, 'try': 0.05555555555555555, 'ry ': 0.05555555555555555, 'y s': 0.05555555555555555, ' so': 0.05555555555555555, 'som': 0.05555555555555555, 'ome': 0.05555555555555555, 'met': 0.05555555555555555, 'eth': 0.05555555555555555, 'thi': 0.05555555555555555, 'hin': 0.05555555555555555, 'ing': 0.05555555555555555, 'ng.': 0.05555555555555555})`

You will store the extracted features in a list that you will call `dataset_small_feat`

In [ ]:
# Write your code here
#dataset_small_feat = list(map(lambda x: x + (count_chars(x[-1]), count_bigrams(x[-1]), count_trigrams(x[-1])), 
#                              dataset_small))

We only compute the unigrams and bigrams as most students have slow machines

In [ ]:
# Write your code here


In [ ]:
dataset_small_feat[:2]

The unigram frequencies

In [ ]:
dataset_small_feat[0][3].items()

The bigram frequencies

In [ ]:
dataset_small_feat[0][4].items()

## Programming: Building $\mathbf{X}$

You will now build the $\mathbf{X}$ matrix. In this assignment, you will only consider unigrams to speed up the training step. This means that you will set aside the character bigrams and trigrams.

When you are done with the lab requirements, feel free to improve the program and include bigrams and trigrams. To add bigrams, a possible method is to add the bigram dictionary to the unigram one using update and then to extract the resulting dictionary. You can easily extend this to trigrams. Feel free to use another method if you want.

In [ ]:
INCLUDE_BIGRAMS = False
if INCLUDE_BIGRAMS:
    for i in range(len(dataset_small_feat)):
        dataset_small_feat[i][3].update(dataset_small_feat[i][4])

### Vectorizing the features

The CLD3 architecture uses embeddings. In this lab, we will simplify it and we will use a feature vector instead consisting of the character frequencies. For example, you will represent the text:

`"Let's try something."`

with:

`{'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 
 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 
 'n': 0.05, 'g': 0.05, '.': 0.05}`

To create the $\mathbf{X}$ matrix, we need to transform the dictionaries of `dataset_small` into numerical vectors. The `DictVectorizer` class from the scikit-learn library, see here [https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html], has two methods, `fit()` and `transform()`, and a combination of both `fit_transform()` to convert dictionaries into such vectors.

You will now write the code to:

1. Extract the character frequency dictionaries from `dataset_small` corresponding to its 3rd index and set them in a list;
2. Convert the list of dictionaries into an $\mathbf{X}$ matrix using `DictVectorizer`.

#### Extracting the character frequencies

Produce a new list of datapoints with the unigrams only. Each item in this list will be a dictionary. You will call it `X_cat`

In [ ]:
# Write your code here


In [ ]:
X_cat[:2]

#### Vectorize `X_cat`

Convert you `X_cat` matrix into a numerical representation using `DictVectorizer`: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html. Call the result `X`.

In [ ]:
# Write your code here


In [ ]:
X[:5]

## Programming: Building $\mathbf{y}$

You will now convert the list of language symbols into a $\mathbf{y}$ vector

Extract the language symbols from `dataset_small_feat` and call the resulting list `y_cat`

In [ ]:
# Write your code here


In [ ]:
y_cat[:5]

Extract the set of language symbols and name it `y_symbols`. Then build two indices mapping the symbols to integers and the integers to symbols. Both indices will be dictionaries that you will call: `lang2idx`and `idx2lang`. Such a conversion is not necessary with sklearn. We do it because many other many machine-learning toolkits (keras or pytorch) require a numerical $\mathbf{y}$ vector and to learn how to carry out this conversion.

In [ ]:
# Write your code here


In [ ]:
idx2lang

In [ ]:
lang2idx

Convert your `y_cat` vector into a numerical vector. Call this vector `y`.

In [ ]:
# Write your code here


In [ ]:
y[:5]

## Programming: Building the Model

Create a neural network using sklearn with a hidden layer of 50 nodes and a relu activation layer: https://scikit-learn.org/stable/modules/neural_networks_supervised.html. Set the maximal number of iterations to 5, in the beginning, and verbose to True. Use the default values for the rest. You will call your classifier `clf`

In [ ]:
# Write your code here


In [ ]:
clf

### Training and Validation Sets

You will now split the dataset into a training and validation sets

#### We shuffle the indices

In [ ]:
indices = list(range(X.shape[0]))
np.random.shuffle(indices)
print(indices[:10])
X = X[indices, :]
y = np.array(y)[indices]

#### We split the dataset
We use a training set of 80% and a validation set of 20%

In [ ]:
training_examples = int(X.shape[0] * 0.8)

X_train = X[:training_examples, :]
y_train = y[:training_examples]

X_val = X[training_examples:, :]
y_val = y[training_examples:]

### Fitting the model

Fit the model on the training set

In [ ]:
# Write your code here


## Predicting

Predict the `X_val` languages. You will call the result `y_val_pred`

In [ ]:
# Write your code here


In [ ]:
y_val_pred[:20]

In [ ]:
y_val[:20]

#### Evaluating

Use the `accuracy_score()` function to evaluate your model on the validation set

In [ ]:
# evaluate the model
accuracy_score(y_val, y_val_pred)

In [ ]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

### Confusion Matrix

In [ ]:
confusion_matrix(y_val, y_val_pred)

You may try to increase the number of iterations to improve the score. You may also try change the parameters of the multilayer percetron.

## Predict the language of a text

Now you will predict the languages of the strings below.

In [ ]:
docs = ["Salut les gars !", "Hejsan grabbar!", "Hello guys!", "Hejsan tjejer!"]

Create features vectors from this list. Call this matrix `X_test`

In [ ]:
# Write your code here


In [ ]:
X_test

And run the prediction that you will store in a variable called `pred_languages`

In [ ]:
# Write your code here


In [ ]:
pred_languages

## Building the Model with Keras
You will now recreate a Keras model with the same architecture as in sklearn.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

### The Model
Create a model identical to the one you created with sklearn. Use the same activation function for the hidden layer and softmax in the last layer.

In [ ]:
# Write your code here


In [ ]:
model

Compile your network with the loss, optimizer, and metrics arguments. As optimizer, use the same as in sklearn. See here: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [ ]:
# Write your code here


Convert the output categories to one-hot vectors. Call the result: `Y_train_cat`

In [ ]:
from keras.utils import to_categorical

In [ ]:
# Write your code here


In [ ]:
Y_train_cat[:5]

Fit your network on your training set. Use two epochs to start with.

In [ ]:
# Write your code here


Evaluate your network on `X_val` and `y_val` with the `evaluate()` method

In [ ]:
# Write your code here


Predict the validation set: `X_val`. Call the result: `Y_val_pred_proba`

In [ ]:
# Write your code here


In [ ]:
Y_val_pred_proba[:5]

Extract the categories from the probabilities in `Y_val_pred_proba`. Use the `np.argmax()` function. Call the result `y_val_pred`. Check that the prediction corresponds to the real values.

In [ ]:
# Write your code here


In [ ]:
y_val_pred[:20]

In [ ]:
y_val[:20]

Print the evaluation

In [ ]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

Print the confusion matrix

In [ ]:
confusion_matrix(y_val, y_val_pred)

Predict your languages with Keras. Reuse `X_test` and call the result `Y_test_pred_proba`.

In [ ]:
# Write your code here


In [ ]:
Y_test_pred_proba

From the probabilities, extract the predicted languages and map them to strings. Call the results `pred_languages_keras`.

In [ ]:
# Write your code here


In [ ]:
pred_languages_keras

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [ ]:
STIL_ID = ["student1", "student1"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "3-language_detector_solution.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the predicted languages.

In [ ]:
ANSWER = json.dumps({'pred_langs': pred_languages})
ANSWER

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [ ]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [ ]:
ASSIGNMENT = 3
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [ ]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
               verify=True)

# from IPython.display import display, JSON
res.json()

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to:
   * Summarize CLD3 and outline its architecture
   * Identify the features used by CLD3
   * Describe your architecture and tell how it is different from CLD3
   * Include the feature matrix you computed manually
   * Outline the differences between sklearn and the neural network API you choose

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is October 1st, 2021.

## Postscript from Pierre Nugues

I created this assignment from an examination I wrote in 2019 for the course on applied machine learning. I simplified it from the `README.md` on GitHub, https://github.com/google/cld3. I found the C++ code difficult to understand and I reimplemented a Keras/Tensorflow version of it from this `README`. Should you be interested, you can find it here: https://github.com/pnugues/language-detector.